# Fields constrain

In [1]:
import warnings
warnings.filterwarnings('ignore')

## The problems
A very common scenario that we face when working with tabular data is finding columns that have very particular relationships between them which are very hard to model and easily confuse the Tabular Models.

Some simple examples of these scenarios include:
- A table that has the columns `country` and `city`: In such scenario, it might be very hard to learn which `country` each `city` belongs to, and when sampling probabilistically, the model is likely to end up generating invalid `country`/`city` combinations.
- A table that contains both the `ag`e and the `date of birth` of a user. The model will learn the `age` and `date of birth` distributions and mostly generate valid combinations, but in some cases it might end up giving back `ages` **that do not correspond to the** `given date of birth`.

These kind of special relationships between columns are called Constraints.


In [2]:
from sdv.demo import load_tabular_demo

employees = load_tabular_demo()
employees

,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Pear,Sales,1,36,35,1,155000.00,22500.00,4,1.0,0.0,0.0
1,Pear,Design,5,37,31,6,44382.47,6490.34,5,0.0,0.0,1.0
2,Glasses,AI,1,45,43,2,134500.00,23500.00,4,1.0,0.0,0.0
3,Glasses,Search Engine,7,47,41,6,132573.15,18931.03,5,0.0,0.0,1.0
4,Cheerper,BigData,6,34,29,5,124000.00,6000.00,2,0.0,1.0,0.0
5,Cheerper,Support,11,33,26,7,56000.00,23500.00,2,0.0,1.0,0.0
6,Pear,Sales,28,43,39,4,98000.00,15000.00,3,1.0,0.0,0.0
7,Pear,Design,75,44,36,8,106133.97,17829.90,1,0.0,0.0,1.0
8,Glasses,AI,33,48,45,3,143500.00,8500.00,3,1.0,0.0,0.0
9,Glasses,Search Engine,56,43,37,6,147053.63,19963.93,4,0.0,0.0,1.0


## How does SDV Handle Constraints?

### Transform Strategy
When using this strategy, SDV applies a transformation to the data before learning it in a way that allows the model to better capture the data properties. For example, if we have one column that needs to be always greater than the other one, SDV can do the following:
- Replace the higher column with the difference between the two columns, which will always be positive.
- Model the transformed data and sample new values.
- Recompute the value of the high column by adding the values of the lower column to it.

The Transform strategy is very efficient and does not affect the speed of the modeling and sampling process, but in some cases might affect the quality of the learning process or simply not be possible.

### Reject Sampling Strategy
In the cases where applying a Transform strategy is not possible or may affect the quality of the learning process, SDV can apply a Reject Sampling strategy.

When using this strategy, SDV validates the sampled rows, discards the ones that do not adjust to the constraint, and re-samples them. This process is repeated until enough rows have been sampled.


## SVD Framework Predefined Constraints

### Unique Constraint

In [3]:
from sdv.constraints import Unique

unique_employee_id_company_constraint = Unique(columns=['employee_id', 'company'])

### UniqueCombinations Constraint

In [4]:
from sdv.constraints import UniqueCombinations

unique_company_department_constraint = UniqueCombinations(columns=['company', 'department'],
                                                          handling_strategy='transform', # 'reject_sampling'
                                                         )

### GreaterThan Constraint

In [5]:
from sdv.constraints import GreaterThan

age_gt_age_when_joined_constraint = GreaterThan(low='age_when_joined',
                                                high='age',
                                                handling_strategy='reject_sampling'
                                               )

**Lower or higher than a scalar value or specific datetime**

In [6]:
salary_gt_30000_constraint = GreaterThan(low=30000,
                                         high=['salary'], # or 'salary'
                                         scalar='low',
                                         handling_strategy='reject_sampling'
                                        )

### Positive and Negative Constraints

In [7]:
from sdv.constraints import Positive

positive_age_constraint = Positive(columns='age',
                                   strict=False,
                                   handling_strategy='reject_sampling'
                                  )

### ColumnFormula Constraint

In [8]:
from sdv.constraints import ColumnFormula

In [9]:
def years_in_the_company(data):
    return data['age'] - data['age_when_joined']

In [10]:
years_in_the_company_constraint = ColumnFormula(column='years_in_the_company',
                                                formula=years_in_the_company,
                                                handling_strategy='transform'
                                               )

### Rounding Constraint

In [11]:
from sdv.constraints import Rounding

salary_rounding_constraint = Rounding(columns='salary',
                                      digits=2,
                                      handling_strategy='transform'
                                     )

### Between Constraint

In [12]:
from sdv.constraints import Between

reasonable_age_constraint = Between(column='age',
                                    low=15,
                                    high=90,
                                    handling_strategy='transform'
                                   )

### OneHotEncoding Constraint

In [13]:
from sdv.constraints import OneHotEncoding

one_hot_constraint = OneHotEncoding(columns=['full_time', 'part_time', 'contractor'])

## Using the CustomConstraint

In [14]:
from sdv.constraints import CustomConstraint

### Apply constraint to one column

In [15]:
def transform(table_data):
    base = 500.
    table_data['salary'] = table_data['salary'] / base
    return table_data

In [16]:
def reverse_transform(table_data):
    base = 500.
    table_data['salary'] = table_data['salary'].round() * base
    return table_data

In [17]:
custome_column_constraint = CustomConstraint(transform=transform, reverse_transform=reverse_transform)

### Constraint to multiple columns

In [18]:
def transform(column_data):
    base = 500.
    return column_data / base

In [19]:
def reverse_transform(column_data):
    base = 500.
    return column_data.round() * base

In [20]:
multiple_columns_constraint = CustomConstraint(columns=['salary', 'annual_bonus'],
                              transform=transform,
                              reverse_transform=reverse_transform
                             )

### Constrain with condition from other columns

In [21]:
def transform(table_data, column):
    base = 500.
    table_data[column] = table_data[column] / base
    return table_data

In [22]:
def reverse_transform(table_data, column):
    base = 500.
    is_not_contractor = table_data["contractor"] == 0.
    table_data[column] = table_data[column].round(4)
    table_data[column].loc[is_not_contractor] = table_data[column].loc[is_not_contractor].round()
    table_data[column] *= base
    return table_data

In [23]:
multiple_columns_with_condition_constraint = CustomConstraint(columns=['salary', 'annual_bonus'],
                                                              transform=transform,
                                                              reverse_transform=reverse_transform
                                                             )

### Constrain based on reject sampling

In [54]:
def is_valid(table_data):
    base = 500.
    return table_data['salary'] % base == 0

def is_valid(column_data):
    base = 500.
    return column_data % base == 0

def is_valid(table_data, column):
    base = 500.
    is_contractor = table_data.contractor == 1
    valid = table_data[column] % base == 0
    contractor_salary = employees['salary'].loc[is_contractor]
    valid.loc[is_contractor] = contractor_salary == contractor_salary.round(2)
    return valid

In [55]:
custom_reject_constraint = CustomConstraint(
                                            columns=['salary', 'annual_bonus'],
                                            is_valid=is_valid
                                           )

## Using the Constraints in Synthesize model

In [51]:
from sdv.tabular import GaussianCopula

constraints = [unique_employee_id_company_constraint,
#                unique_company_department_constraint,
#                age_gt_age_when_joined_constraint,
#                salary_gt_30000_constraint,
#                positive_age_constraint,
#                years_in_the_company_constraint,
#                salary_rounding_constraint,
#                reasonable_age_constraint,
#                one_hot_constraint,
#                multiple_columns_with_condition_constraint,
               custom_reject_constraint,
              ]

model = GaussianCopula(constraints=constraints)

In [52]:
model.fit(employees)

ConstraintsNotMetError: Data is not valid for the given constraints

In [42]:
sampled = model.sample(100)

AttributeError: 'NoneType' object has no attribute 'sample'

In [30]:
sampled

,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Glasses,AI,38,46,41,5,78000.00,21000.00,4,1.0,0.0,0.0
1,Cheerper,BigData,8,46,40,6,138000.00,7500.00,2,1.0,0.0,0.0
2,Glasses,Search Engine,16,40,34,6,117340.60,8195.65,2,0.0,0.0,1.0
3,Pear,Design,70,33,27,6,44382.45,21614.00,4,0.0,0.0,1.0
4,Cheerper,Support,43,45,38,7,155000.00,23500.00,2,0.0,1.0,0.0
5,Cheerper,BigData,42,35,30,5,137000.00,8500.00,2,0.0,1.0,0.0
6,Glasses,Search Engine,5,43,37,6,155000.00,7086.60,4,0.0,0.0,1.0
7,Pear,Sales,8,47,43,4,151500.00,12500.00,4,1.0,0.0,0.0
8,Cheerper,BigData,2,33,27,6,100500.00,23000.00,1,0.0,1.0,0.0
9,Pear,Design,5,33,28,5,104500.00,21000.00,2,0.0,1.0,0.0
